In [3]:
#Password to get in is "root"
import sqlite3
import time
conn = sqlite3.connect("Vending.db")
cursor = conn.cursor()
vending_dict = {"Lace":10, "Ringo":12, "Kirukure":8, "Pepsea":20, "Cocalola":25, "Orio":15, "Kittykat":5, "Dirymilk":8, "Fonta":30}

In [2]:
#Creation of the tables
conn.execute('''
CREATE TABLE accounts(
name TEXT PRIMARY KEY NOT NULL,
balance INTEGER NOT NULL
);
''')
conn.execute('''
CREATE TABLE stock(
slot INTEGER PRIMARY KEY NOT NULL,
remaining INTEGER NOT NULL
);
''')
conn.execute('''
CREATE TABLE sales(
item TEXT PRIMARY KEY NOT NULL,
profit INTEGER NOT NULL,
unit INTEGER NOT NULL
);
''')
conn.execute("INSERT INTO stock VALUES(1,0), (2,0), (3,0), (4,0), (5,0), (6,0), (7,0), (8,0),(9,0);")
conn.execute("INSERT INTO sales VALUES('Lace',0,0), ('Ringo',0,0), ('Kirukure',0,0), ('Pepsea',0,0), ('Cocalola',0,0), ('Orio',0,0), ('Kittykat',0,0), ('Dirymilk',0,0),('Fonta',0,0);")
conn.commit()

In [ ]:
#To delete all rows from a table
#conn.execute("DELETE FROM stock;")

#To delete a table
#conn.execute("DROP TABLE stock;")

In [13]:
# @title
#All functions
def print_machine():
  print('''
  __________________________________________________
 |==================================================|
 |                                                  |
 |             [====================]               |
 |                                                  |
 |                                                  |
 |                                                  |
 |                                                  |
 |                                                  |
 |                                                  |
 |                                                  |
 +--------------------------------------------------+
  ''')
  time.sleep(1)
  print('''
  __________________________________________________
 |                                                  |
 |________/==================================================//
 |       /                                                  //
 |      /               [====================]             //
 |     /                                                  //
 |    /                                                  //
 |   /                                                  //
 |  /                                                  //
 | /                                                  //
 |/                                                  //
 +--------------------------------------------------+/
  ///////////////////////////////////////////////////
  ''')

def print_out(item):
  print('''
|==                                                         ==|
|==                                                         ==|
|==                  __________________                     ==|
|==                 /                  \\                    ==|
|==                /                    \\                   ==|
|==               |        ______        |                  ==|
|==               |       /*****/        |                  ==|
|==               |      +-----+         |                  ==|
|==                \\                    /                   ==|
|==                 \\__________________/                    ==|
|==                                                         ==|
|==                                                         ==|
|==                                                         ==|
''')
  print(f"Here you have your {item}")

def add_acc():
  print("\nGoing to create account")
  name = input("Enter you name that will act as you account name, and you can access through this name:")
  balance = int(input("Enter the initial balance:"))
  conn.execute("INSERT INTO accounts VALUES(?,?);",(name,balance))
  conn.commit()
  print(f"Account has been created for {name}:{balance}")

def check_acc():
  ip_name = input("Enter account name:")
  cursor.execute("SELECT * FROM accounts WHERE name = ?;",(ip_name,))
  result = cursor.fetchall()
  if result:
    return result
  else:
    while(1):
      choice = input("Account not found, would you like to create a new account Y/N:")
      if choice.lower() == "y":
        add_acc()
        break
      elif choice.lower() == "n":
        print("No account needed")
        break
      else:
        print("Invalid input")

def stock_fill():
  while(1):
    print("\n1. Fill all\n2. Fill one slot only\n3. Exit filling")
    ip = int(input("Enter you choice:"))
    if ip == 1:
      conn.execute("UPDATE stock SET remaining = 10")
      conn.commit()
      print("All slots filled with max capacity, that is 10 quantity")
      return
    elif ip == 2:
      ip_slot = int(input("Enter the slot:"))
      if(ip_slot>0 and ip_slot<=9):
        ip_quantity = int(input("Enter the quantity:"))
        if(ip_quantity<=0):
          print("The quantity provided is 0 or negative")
        else:
          cursor.execute("SELECT remaining FROM stock WHERE slot =?", (str(ip_slot)))
          in_stock = cursor.fetchall()
          if (in_stock[0][0] + ip_quantity)>10:
            print("Maximum limit has been overflowed for the given quantity, Maximum quatity stored for now")
            conn.execute("UPDATE stock SET remaining = 10 WHERE slot = ?",(str(ip_slot)))
            conn.commit()
          else:
            conn.execute("UPDATE stock SET remaining = remaining + ? WHERE slot = ?",(str(ip_quantity),str(ip_slot)))
            conn.commit()
            print(f"Stock refilled for slot {ip_slot} and the quatity remaining is {in_stock[0][0]+ip_quantity}")
      else:
        print("Invalid slot number")
      return
    elif ip == 3:
      return
    else:
      print("Invalid choice")

def stock_check_reduce(slot):
  cursor.execute("SELECT remaining FROM stock WHERE slot = ?",(str(slot)))
  slot_value = cursor.fetchall()
  if slot_value[0][0] > 0:
    if slot_value[0][0]<4:
      if slot_value[0][0] == 1:
        print(f"! Warning, no more stock left after purchase")
      else:
        print(f"! Warning, more {(slot_value[0][0])-1} stock left after purchase")
    conn.execute("UPDATE stock SET remaining = remaining - 1 WHERE slot = ?",(str(slot)))
    conn.commit()
    return True
  else:
    return False

def stock_check():
  cursor.execute("SELECT remaining FROM stock")
  slot_values = cursor.fetchall()
  for i in range(len(slot_values)):
    print(f"Slot:{i+1} - {slot_values[i][0]}")

def sales_check():
  cursor.execute("SELECT * FROM sales;")
  sales = cursor.fetchall()
  for j in sales:
    print(f"{j[0]}-{j[1]}-{j[2]}")

def buy(item,acc,slot):
  balance = acc
  print(item, vending_dict[item])
  cost = vending_dict[item]
  if balance:
    if cost>balance[0][1]:
      print("Insufficient balance")
    else:
      if stock_check_reduce(slot) == True:
        print_out(item)
        conn.execute("UPDATE sales SET profit = profit + ?, unit = unit + 1 WHERE item = ?", (str(vending_dict[item]), item))
        conn.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?", (cost, balance[0][0]))
        conn.commit()
        cursor.execute("SELECT balance FROM accounts WHERE name =?",(balance[0][0],))
        remain = cursor.fetchall()
        print(f"Your balance is:{remain[0][0]}")
      else:
        print(f"{item} is out of stock")

In [14]:
#To check what values are there in the tables

cursor.execute("SELECT * FROM accounts;")
test = cursor.fetchall()
print(test)
cursor.execute("SELECT * FROM stock;")
stock = cursor.fetchall()
print(stock)
cursor.execute("SELECT * FROM sales;")
sales = cursor.fetchall()
print(sales)

[('Me', 0), ('Kk', 30)]
[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0)]
[('Lace', 70, 7), ('Ringo', 60, 5), ('Kirukure', 40, 5), ('Pepsea', 0, 0), ('Cocalola', 0, 0), ('Orio', 0, 0), ('Kittykat', 0, 0), ('Dirymilk', 0, 0), ('Fonta', 0, 0)]


In [12]:
# @title
#Main vending machine code
while(1):
  print("\nMenu\n1. Buy\n2. Create account\n3. Inventory management\n4. Exit")
  ip = int(input("Enter your choice number:"))
  if ip ==1:
    print('''
    +-----------------------------------------------------+
    |                                                     |
    |                                                     |
    |                  Insert you card                    |
    |_____________________________________________________|
    |            |__________________________|             |
    |            |   ____________________   |             |
    |            |__|--------------------|__|             |
    |               |  Meow credit card  |/               |
    |               |         xxxx-xxxx  |/               |
    |               |____________________|/               |
    |               ///////////////////////               |
    |                                                     |
    |                                                     |
    |                                                     |
    +-----------------------------------------------------+''')
    time.sleep(1)
    print('''
    +-----------------------------------------------------+
    |                                                     |
    |                                                     |
    |                  Insert you card                    |
    |_____________________________________________________|
    |            |__|--------------------|/_|             |
    |            |  |  Meow credit card  |/ |             |
    |            |__|         xxxx-xxxx  |/_|             |
    |               |____________________|/               |
    |               ///////////////////////               |
    |                                                     |
    |                                                     |
    |                                                     |
    |                                                     |
    |                                                     |
    +-----------------------------------------------------+''')
    see_acc = check_acc()
    if see_acc:
      while(1):
        print("\n=======================================================")
        print("|==++==++=========+-Vending Machine-+=========++==++==|")
        print("|==                                                 ==|")
        print("|==    ___________    ___________    ___________    ==|")
        print("|==   |           |  |           |  |           |   ==|")
        print("|==   |   Lace    |  |   Ringo   |  |  Kirukure |   ==|")
        print("|==   |    ₹10    |  |    ₹12    |  |     ₹8    |   ==|")
        print("|==   \\___________/  \\___________/  \\___________/   ==|")
        print("|==         1              2              3         ==|")
        print("|==    ___________    ___________    ___________    ==|")
        print("|==   |           |  |           |  |           |   ==|")
        print("|==   |   Pepsea  |  |  Cocalola |  |    Orio   |   ==|")
        print("|==   |    ₹20    |  |    ₹25    |  |    ₹15    |   ==|")
        print("|==   \\___________/  \\___________/  \\___________/   ==|")
        print("|==         4               5             6         ==|")
        print("|==    ___________    ___________    ___________    ==|")
        print("|==   |           |  |           |  |           |   ==|")
        print("|==   |  Kittykat |  |  Dirymilk |  |   Fonta   |   ==|")
        print("|==   |     ₹5    |  |     ₹8    |  |    ₹30    |   ==|")
        print("|==   \\___________/  \\___________/  \\___________/   ==|")
        print("|==         7               8             9         ==|")
        print("|==-------------------------------------------------==|")
        print("|==                 ______________                  ==|")
        print("|==                /              \\                 ==|")
        print("|==               |                |                ==|")
        print("|==                \\______________/                 ==|")
        print("|==                                                 ==|")
        print("|==-------------------------------------------------==|")
        print("|=====================================================|")
        print("=======================================================")
        print(f"You have {see_acc[0][1]} amount balance")
        slot = int(input("\nEnter you choice slot number or \"0\" for main menu:"))

        if slot == 0:
          break
        elif slot == 1:
          item = "Lace"
        elif slot == 2:
          item = "Ringo"
        elif slot == 3:
          item = "Kirukure"
        elif slot == 4:
          item = "Pepsea"
        elif slot == 5:
          item = "Cocalola"
        elif slot == 6:
          item = "Orio"
        elif slot == 7:
          item = "Kittykat"
        elif slot == 8:
          item = "Dirymilk"
        elif slot == 9:
          item = "Fonta"
        else:
          print("Enter a valid slot number or enter \"0\" to exit and go to the main menu")
        buy(item,see_acc,slot)
        break

  elif ip == 2:
    add_acc()

  elif ip == 3:
    psw = input("\nEnter the password to open the machine:")
    if psw == "root":
      print("You are in")
      print_machine()
      while(1):
        print("\n1. Stock check\n2. Stock refill\n3. Sales check\n4. Exit inventory")
        choice = int(input("Enter your choice:"))
        if choice == 1:
          stock_check()
        elif choice == 2:
          stock_fill()
        elif choice == 3:
          sales_check()
        elif choice == 4:
          break
        else:
          print("Invalid choice")
    else:
      print("Invalid password")

  elif ip == 4:
    break

  else:
    print("Invalid menu input")


Menu
1. Buy
2. Create account
3. Inventory management
4. Exit
Enter your choice number:1

    +-----------------------------------------------------+
    |                                                     |
    |                                                     |
    |                  Insert you card                    |
    |_____________________________________________________|
    |            |__________________________|             |
    |            |   ____________________   |             |
    |            |__|--------------------|__|             |
    |               |  Meow credit card  |/               |
    |               |         xxxx-xxxx  |/               |
    |               |____________________|/               |
    |               ///////////////////////               |
    |                                                     |
    |                                                     |
    |                                                     |
    +----